# PrepareData.ipynb

## Read data from downloaded csv files and create dataframes.

NOTE: Company_Tweet.csv & Tweet.csv are very large data files could not upload them in git as account limit is max 100MB per file

Below are the steps followed to read data and store dataframes:
* To run this code download Resources Company_Tweet.csv & Tweet.csv from kaggle link https://www.kaggle.com/code/saadusama/twitter-s-impact-on-stock-market-prices/data and copy them in Resources folder
* Read tickers data from Resources/CompanyValues.csv, filter Tesla stock data and store it in a dataframe
* Read Twitter data from Company_Tweet.csv & Tweet.csv, filter tweets for Tesla and store in a dataframe

### Necessary imports

In [2]:
import pandas as pd
from pathlib import Path
from datetime import datetime
import pandas_ta as ta

### Read Ticker data
* Read stock data from Resources/CompanyValues.csv
* Filter dataframe to store only TSLA data
* Drop ticker_symbol column as it is not required anymore
* set index to day_date
* Review DataFrame

In [7]:
# NOTE: Discovered during analysis that the data downloaded from Kaggle seems to be different from the data available on Web, 
# it seems data in kaggle is pre-processed before saving, hence downloaded stock data using GOOGLEFINANCE function and saved in TSLA.csv

# market_df = pd.read_csv(Path('./Resources/CompanyValues.csv'), index_col='day_date', parse_dates=True, infer_datetime_format=True)
# tsla_stock_values_df = market_df[market_df["ticker_symbol"] == "TSLA"].dropna()
# tsla_stock_values_df = tsla_stock_values_df.drop(["ticker_symbol"],axis=1)

tsla_stock_values_df = pd.read_csv(Path('./Resources/TSLA.csv'), index_col='Date', parse_dates=True, infer_datetime_format=True)

# renaming columns so that existing code is not impacted
tsla_stock_values_df.rename({"Date": "day_value", "High": "high_value",
                                       "Low":"low_value", "Open":"open_value",
                                       "Close":"close_value",
                                       "Volume":"volume"},axis=1)
tsla_stock_values_df = tsla_stock_values_df.sort_index()
tsla_stock_values_df = tsla_stock_values_df.loc['2015-01-01':]
# tsla_stock_values_df.head()

,high_value,low_value,open_value,close_value,volume,Adj Close
Date,,,,,,
2015-01-02,14.883333,14.217333,14.858000,14.620667,71466000,14.620667
2015-01-05,14.433333,13.810667,14.303333,14.006000,80527500,14.006000
2015-01-06,14.280000,13.614000,14.004000,14.085333,93928500,14.085333
2015-01-07,14.318667,13.985333,14.223333,14.063333,44526000,14.063333
2015-01-08,14.253333,14.000667,14.187333,14.041333,51637500,14.041333


### Read Twitter Data and prepare one DataFrame for TSLA tweets
* Read Tweets from Resources/Tweet.csv and review dataframe
* Read Resources/Company_Tweet.csv, to find tweets relevant for TSLA, and review dataframe
* Merge both dataframes on tweet_id to get the consolidated tweet data for TSLA
* Review merged dataframe
* Convert post_date to Datetime format

In [ ]:
# Read tweets & company_twee data from csv
tweets_df = pd.read_csv(Path("Resources/Tweet.csv"))
company_tweets_df = pd.read_csv(Path("Resources/Company_Tweet.csv"))

# Filter TSLA tweets
tsla_tweets_df = company_tweets_df[company_tweets_df["ticker_symbol"] == "TSLA"]
tsla_tweets_df = pd.merge(tsla_tweets_df,tweets_df,on="tweet_id")

# Compute total_engagement for a tweet, and filter tweets that have total_engagement > 2
# If we take total_engagment > 3 or more we are losing day data, hence for better training models kept a threshold of 2
tsla_tweets_df["total_engagement"] = tsla_tweets_df["comment_num"] + tsla_tweets_df["retweet_num"] + tsla_tweets_df["like_num"]
tsla_tweets_df = tsla_tweets_df[tsla_tweets_df["total_engagement"] > 2]

# Convert post_date from int64 to datetime type
tsla_tweets_df.post_date=tsla_tweets_df.post_date.apply(lambda z:datetime.fromtimestamp(z))

### Store dataframes in IPython's Database
This will help reuse the data, without repeating the code.
* %store - stores variables, aliases and macros in IPython’s database.
* store TSLA tweets dataframe and stock market data in IPython's database
* Run Sentiment Analysis and store results in IPython's database

In [ ]:
# Prepare stock dataframe with Technical Indicatoras

ta_df = tsla_stock_values_df.copy()

#Create daily_return column
ta_df['daily_return']= ta_df['close_value'].pct_change()

#Set up new column 'price_direction', where 0 if 'daily_return' < 0, 
#and 1 if 'daily_return' > 0. 
ta_df['price_direction']= 0
ta_df.loc[ta_df['daily_return'] >= 0, 'price_direction'] =1

# Calculate MACD values using the pandas_ta library
ta_df.ta.macd(close='close_value', fast=12, slow=26, signal=9, append=True)

# Calculate High-Low Percentage values using the pandas_ta library
ta_df['HL_PCT'] = (ta_df['high_value'] - ta_df['low_value']) / ta_df['close_value'] * 100.0

# Calculate RSI values using the pandas_ta library
ta_df['RSI'] = ta_df.ta.rsi(close= 'close_value', length= 14, scalar= 100)

# Calculate VPT values using the pandas_ta library
ta_df['PVT'] = ta_df.ta.pvt(close= 'close_value',volume= 'volume',drift= 1)

# Clean Nan
ta_df= ta_df.dropna()

ta_df=ta_df.drop(['daily_return','volume','open_value','high_value','low_value'],
           axis=1)
ta_df.head()


In [ ]:
# save filtered tweets data
%store tsla_tweets_df

# save stock data read from web
%store tsla_stock_values_df

# store technical indicators data frame
# technical indicators are computed based on available data - these bascially forms the features matrix(X) for machine learning model
%store ta_df

In [ ]:
# run SentimentAnalysis.ipynb, this notebook contains code for calculating polarity based on tweets using VaderSentiment Analysis, 
# the result of the analysis is stored dataframe tsla_sentiments_df
%run SentimentAnalysis.ipynb

# run Textblob_SA.ipynb, this notebook contains code to analyse tweets using TextBlobs
# the result of the analysis is stored dataframe tsla_sentiments_df_textblob
%run Textblob_SA.ipynb